In [12]:
import glob
import pandas as pd

def filter_and_validate_dates(file_pattern, starting_date, ending_date, segment, index):
    # Loading all CSV files matching the pattern
    csv_files = glob.glob(file_pattern)
    dataframes = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df.rename(columns={"Date": "date"}, inplace=True)
            df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")
            dataframes.append(df)
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    # Combining all dataframes into one
    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_df['date'] = pd.to_datetime(combined_df['date'])

    # Generating all dates in the range
    all_dates = pd.date_range(start=starting_date, end=ending_date, freq='D')

    # Filtering data by date range and ticker
    filtered_by_date_range = combined_df[(combined_df['date'] >= pd.to_datetime(starting_date)) & 
                                         (combined_df['date'] <= pd.to_datetime(ending_date))]
    filtered_by_ticker = filtered_by_date_range[
        filtered_by_date_range['Ticker'].str.contains(segment, case=False) & 
        filtered_by_date_range['Ticker'].str.contains(index, case=False)]

    # Validating each date in the range
    date_exists = {date.strftime("%Y-%m-%d"): date in filtered_by_date_range['date'].values for date in all_dates}

    return filtered_by_ticker, date_exists

# Inputs
file_pattern = "*_BACKADJUSTED_*.csv"
starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
segment = input("Enter the segment (e.g., BFO/NFO/MCX): ")
index = input("Enter the index (e.g., CRUDEOIL, GOLD): ")



Enter the starting date (YYYY-MM-DD): 2023-04-03
Enter the ending date (YYYY-MM-DD): 2023-04-30
Enter the segment (e.g., BFO/NFO/MCX): MCX
Enter the index (e.g., CRUDEOIL, GOLD): CRUDEOIL


In [13]:
# Function execution to get Filtered Data
data, date_check = filter_and_validate_dates(file_pattern, starting_date, ending_date, segment, index)

print(data)
print("Date Existence Check:")
for date, exists in date_check.items():
    print(f"{date}: {'Exists' if exists else 'Does not exist'}")

                         Ticker       date      Time   Close    Spot  Open  \
0       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:02:59     3.0     0.0   NaN   
1       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:22:59     2.9     0.0   NaN   
2       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:26:59     2.7     0.0   NaN   
3       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:46:59     2.8     0.0   NaN   
4       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:53:59     2.8     0.0   NaN   
...                         ...        ...       ...     ...     ...   ...   
538508        CRUDEOILM-III.MCX 2023-04-17  23:06:59  6667.0  6700.0   NaN   
538509        CRUDEOILM-III.MCX 2023-04-17  23:09:59  6663.0  6700.0   NaN   
538510        CRUDEOILM-III.MCX 2023-04-17  23:16:59  6670.0  6700.0   NaN   
538511        CRUDEOILM-III.MCX 2023-04-17  23:17:59  6671.0  6700.0   NaN   
538512        CRUDEOILM-III.MCX 2023-04-17  23:27:59  6663.0  6700.0   NaN   

        High  Low  Volume  Open Interest  Unnamed: 9  Unnamed: 

In [14]:
# Creating a list of dates where data exists
existing_dates = [date for date, exists in date_check.items() if exists]

# Printing the list of existing dates
print("Existing Dates:")
print(existing_dates)


Existing Dates:
['2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-10', '2023-04-11', '2023-04-12', '2023-04-13', '2023-04-14', '2023-04-17', '2023-04-18', '2023-04-19', '2023-04-20', '2023-04-21', '2023-04-24', '2023-04-25', '2023-04-26', '2023-04-27', '2023-04-28']


In [21]:
from datetime import datetime
import pandas as pd
import re  

months = {
    "JAN": 1, "FEB": 2, "MAR": 3, "APR": 4,
    "MAY": 5, "JUN": 6, "JUL": 7, "AUG": 8,
    "SEP": 9, "OCT": 10, "NOV": 11, "DEC": 12
}


# Assuming this is the function from previous examples
def get_all_expiry_dates(data, input_date, index, segment):
    input_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    year = input_date_obj.year
#     month_abbr = input_date_obj.strftime("%b").upper()
#     regex_pattern = f"{index}(\d{{2}}){month_abbr}.*?\.{segment}"
    regex_pattern = f"{index}(\d{{2}}).*?\.{segment}"
    filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]
    expiry_days = set()
    
#     print(filtered_by_ticker['Ticker'])
    delta=12-input_date_obj.month
    for ticker in filtered_by_ticker['Ticker']:
        try:
#         match = re.search(f"{index}(\d{{2}}){month_abbr}", ticker)

#         match = re.search(f"{index}(\d{{2}})", ticker)
          match = ticker.split(index)[1][:5]
          day_str = match[:2]
          month_str = match[2:]
          
          expiry_date = datetime(year,months[month_str],int(day_str) )
#           if(input_date_obj.month==months[month_str]):
#                 expiry_days.add(expiry_date)
#           elif((expiry_date-input_date_obj).days >=0):
#                 expiry_days.add(expiry_date)
                
         # if we want just next
          if((expiry_date-input_date_obj).days >=0):
                expiry_days.add(expiry_date)
        except ValueError:
            continue


    return sorted(expiry_days)


# Dictionary to store expiry dates for each date
all_expiry_dates = {}

# Loop through each existing date and fetch expiry dates
for date in existing_dates:
    expiries = get_all_expiry_dates(data, date, index, segment)
    all_expiry_dates[date] = [expiry_date.strftime("%Y-%m-%d") for expiry_date in expiries]

# Printing all expiry dates for each date
print("\nAll Expiry Dates:")
for date, expiries in all_expiry_dates.items():
    print(f"{date}: {expiries}") 

/var/folders/xm/_6m30k1j1kbgb2ps1nz6t4d00000gn/T/ipykernel_58856/503444725.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]


{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), datetime.datetime(2023, 4, 23, 0, 0)}
{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), datetime.datetime(2023, 4, 23, 0, 0)}
{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), datetime.datetime(2023, 4, 23, 0, 0)}
{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), datetime.datetime(2023, 4, 23, 0, 0)}
{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), datetime.datetime(2023, 4, 23, 0, 0)}
{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), datetime.datetime(2023, 4, 23, 0, 0)}
{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), datetime.datetime(2023, 4, 23, 0, 0)}
{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), datetime.datetime(2023, 4, 23, 0, 0)}
{datetime.datetime(2023, 5, 23, 0, 0), datetime.datetime(2023, 6, 23, 0, 0), dat

In [25]:
# Re-importing the necessary module and redefining the function as the execution state was reset
import re

# Load the dataset again
data_path = '/mnt/data/GFDLMCX_BACKADJUSTED_03042023 2.csv'
data = pd.read_csv(data_path)

# Redefine the function to specifically target CRUDEOIL index and MCX segment with adjusted criteria
def extract_crudeoil_date_month(ticker):
    # Check for specific index and segment
    if "CRUDEOIL" in ticker and ticker.endswith(".MCX"):
        # The expected format is CRUDEOIL{date}{month}{selected_strike}{option_type}.MCX
        # Example: CRUDEOIL23APR4500PE.MCX
        # Strip away the index and segment parts to isolate the middle portion
        main_part = ticker.replace("CRUDEOIL", "").replace(".MCX", "")
        # Use regex to find the pattern {date}{month}{selected_strike}{option_type}
        match = re.search(r'(\d{2})([A-Z]{3})\d{4}(CE|PE)', main_part)
        if match:
            # Extract the date and month from the match groups
            date = match.group(1)
            month = match.group(2)
            return date, month
    return None

# Apply the adjusted function to all tickers in the dataset
crudeoil_date_month = [extract_crudeoil_date_month(ticker) for ticker in data['Ticker']]
# Get unique date and month combinations
unique_crudeoil_date_month = set([dm for dm in crudeoil_date_month if dm is not None])
unique_crudeoil_date_month


{('23', 'APR'), ('23', 'JUN'), ('23', 'MAY')}

In [26]:
# Updated function to extract and verify date-month pairs against a list of existing dates
def extract_date_month(ticker, index, segment, existing_dates):
    if ticker.startswith(index) and ticker.endswith(segment):
        main_part = ticker[len(index):-len(segment)-1]  # Exclude the dot in the segment
        match = re.search(r'(\d{2})([A-Z]{3})\d{4}(CE|PE)', main_part)
        if match:
            day = int(match.group(1))
            month = match.group(2)
            year = datetime.now().year  # Assuming current year; adjust as necessary
            ticker_date_str = f"{year}-{month}-{day:02d}"
            ticker_date = datetime.strptime(ticker_date_str, "%Y-%b-%d")
            
            # Check if ticker_date is in the list of existing dates
            if ticker_date in existing_dates:
                return (day, month)
    return None

In [30]:
import pandas as pd
import re
from datetime import datetime

# Load the dataset
data_path = 'GFDLMCX_BACKADJUSTED_03042023 2.csv'
data = pd.read_csv(data_path)


# Generalized function to extract date and month from a ticker string based on given index and segment
def extract_date_month(ticker, index, segment):
    # Format is {index}{date}{month}{selected_strike}{option_type}.{segment}
    # Example: CRUDEOIL23APR4500PE.MCX
    if ticker.startswith(index) and ticker.endswith(segment):
        # Remove index and segment to isolate the middle portion
        main_part = ticker[len(index):-len(segment)-1]  # Exclude the dot in the segment
        # Regex to find the pattern {date}{month}{selected_strike}{option_type}
        match = re.search(r'(\d{2})([A-Z]{3})\d{4}(CE|PE)', main_part)
        if match:
            # Extract the date and month
            date = match.group(1)
            month = match.group(2)
            return date, month
    return None

# Example usage: extracting dates and months for CRUDEOIL tickers in the MCX segment
date_month_info = [extract_date_month(ticker, index, segment) for ticker in data['Ticker']]
unique_date_month_info = set([info for info in date_month_info if info is not None])

# Displaying unique date and month combinations
print(unique_date_month_info)


{('23', 'MAY'), ('23', 'APR')}


In [31]:
import pandas as pd
import re
from datetime import datetime

# Generalized function to extract date and month from a ticker string based on given index, segment, and date check
def extract_date_month(ticker, index, segment, existing_dates):
    # Format is {index}{date}{month}{selected_strike}{option_type}.{segment}
    # Example: CRUDEOIL23APR4500PE.MCX
    if ticker.startswith(index) and ticker.endswith(segment):
        # Remove index and segment to isolate the middle portion
        main_part = ticker[len(index):-len(segment)-1]  # Exclude the dot in the segment
        # Regex to find the pattern {date}{month}{selected_strike}{option_type}
        match = re.search(r'(\d{2})([A-Z]{3})\d{4}(CE|PE)', main_part)
        if match:
            # Extract the date and month
            day = int(match.group(1))
            month = match.group(2)
            year = datetime.now().year  # Assuming current year; adjust as necessary if required
            # Create a datetime object to compare against existing_dates
            extracted_date = datetime.strptime(f"{day} {month} {year}", "%d %b %Y")

            # Check if extracted_date is in the list of existing dates
            if extracted_date in existing_dates:
                return day, month
    return None

date_month_info = [extract_date_month(ticker, index, segment, existing_dates) for ticker in data['Ticker']]
unique_date_month_info = set([info for info in date_month_info if info is not None])

# Displaying unique date and month combinations that match the existing dates
print(unique_date_month_info)


set()


now we will create a function which take date which Exists, segment, index, expiry type, DTE {for monthly/weekly} 

if daily expiry then see ticker named 




filtered_date_to_work = f(existing_date, type_of_expiry, DTE)

if type_of_expiry is daily:
   Date is existing_date and Ticker Date consists of string existing_date {in daily DTE is not defined}
   
if type_of_expiry is weekly:
   put that existing_date inside a function which tell me all existing expiry of that month then see what week we need to work like week 1 , week 2 then we will also take DTE as input also i.e. week 1 DTE 1 means we need to go to week 1 then go to 

all_expiry_of_month = f(one of existing_date)
   This function tell me all expiry of this month
   
Make a List of Existing Date
2023-04-03: Exists
2023-04-04: Exists
2023-04-05: Exists
2023-04-06: Exists
2023-04-07: Does not exist
2023-04-08: Does not exist
2023-04-09: Does not exist

and convert to 
2023-04-03: Exists
2023-04-04: Exists
2023-04-05: Exists
2023-04-06: Exists

Take input of type_of_expiry, DTE

example:
  if type_of_expiry is Daily then we will see our all_expiry_of_month and find nearest element to this date ... "date in expiry - daily date" should be min ...store that date in ticker_date list
  
  if type_of_expiry is Weekly we will do a + DTE to the input Date in our List and see if the Date we get to is same as that week expiry or Not, if not we will move to next possible existing date else if we we will store that date in ticker_date list

